# Deter Model training



In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
from pathlib import Path, PurePath
import matplotlib.pyplot as plt
import glob


sys.path.insert(0, "/home/haridas/projects/opensource/detr")
sys.path.insert(0, "../")
os.environ["CUDA_VISIBLE_DEVICES"] = ""

%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
from mystique.utils import plot_results

In [ ]:

import torch
from torch.utils.data import DataLoader, SequentialSampler
import torchvision.transforms as T
import torchvision.transforms.functional as F
from PIL import Image

import datasets
from datasets import build_dataset, get_coco_api_from_dataset
from datasets.coco_eval import CocoEvaluator
from datasets.coco import make_coco_transforms

from models.detr import DETR, SetCriterion, PostProcess
from models.transformer import build_transformer
from models.backbone import build_backbone
from models.matcher import build_matcher
from engine import evaluate

from util.misc import collate_fn, NestedTensor
from util.plot_utils import plot_logs, plot_precision_recall

from datasets.coco import make_coco_transforms

In [ ]:
# img_transform = make_coco_transforms("val")
transform = T.Compose([
    T.Resize(800),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
T.Resize(800)

## Strip Trained model for transfer learning

In [ ]:
# checkpoint = torch.load(f"{basedir}/detr-r50-e632da11.pth", map_location='cpu')
# checkpoint = torch.load(f"{basedir}/detr-r101-dc5-a2e86def.pth", map_location='cpu')

In [ ]:
# checkpoint["model"].keys()

In [ ]:
# # Param sets that needs to be custom learned.
# del checkpoint["model"]["class_embed.weight"]
# del checkpoint["model"]["class_embed.bias"]
# del checkpoint["model"]["query_embed.weight"]

In [ ]:
# torch.save(checkpoint, f"{basedir}/detr-r101-dc5-a2e86def-class-head.pth")

## Dataset 

In [ ]:
class Args:
    coco_path = "/home/haridas/projects/mystique/data/train_and_test-2020-Jun-05-coco/"
    dataset_file = "pic2card"
    masks = False
    
train_ds = datasets.custom_coco_build("train", Args)

In [ ]:
image, target = super(datasets.coco.CocoDetection, train_ds).__getitem__(10)
target = {'image_id': train_ds.ids[0], 'annotations': target}
image, target = train_ds.prepare(image, target)

In [ ]:
target

In [ ]:
_image, _target = datasets.transforms.RandomResize([800], max_size=1333)(image, target)
_target

In [ ]:
# torch.rand()
# region = T.RandomCrop.get_params(image, (799, 1000))

In [ ]:
# F.crop(image, *T.RandomCrop.get_params(image, (400, 300)))
# image.shape

In [ ]:
# datasets.transforms.crop(image, target, )

In [ ]:
# datasets.transforms.crop

In [ ]:
# np.asarray(img)

In [ ]:
# image.permute(1, 2, 0).numpy()
# image.permute(1, 2, 0).numpy()

In [ ]:
# Image.fromarray(image)

In [ ]:
# The index are directly from the coco dataset index.
CLASSES = {
     0: 'background', # This one is a default class learned by model, or a catch all.
     1: 'textbox',
     2: 'radiobutton',
     3: 'checkbox',
     4: 'actionset',
     5: 'image',
     6: 'rating'
}

## Generate Coco Metrics

In [ ]:
class DefaultConf:
    # Basic network
    backbone = "resnet50"
    position_embedding = "sine"
    hidden_dim = 256
    dropout = 0.1
    nheads = 8
    dim_feedforward = 2048
    enc_layers = 6
    dec_layers = 6
    pre_norm = False
    num_queries = 100
    aux_loss = False
    
    # Force to eval model
    lr_backbone = 0
    masks = False
    dilation = False
    device = "cuda"
    
    # Loss tuning params.
    set_cost_class = 1
    set_cost_bbox = 5
    set_cost_giou = 2
    bbox_loss_coef = 5
    giou_loss_coef = 2
    eos_coef = 0.1
    losses = ["labels", "boxes", "cardinality"]

    # Configuration fitting the pic2card specific
    # class configuration.
    coco_path = "/home/haridas/projects/mystique/data/train_and_test-2020-Jun-05-coco/"
    dataset_file = "pic2card"

weight_dict = {
    'loss_ce': 1,
    'loss_bbox': DefaultConf.bbox_loss_coef,
    'loss_giou': DefaultConf.giou_loss_coef
}
    
backbone = build_backbone(DefaultConf)

transformer_network = build_transformer(DefaultConf)
matcher = build_matcher(DefaultConf)
criterion = SetCriterion(num_classes=len(CLASSES),
                         matcher=matcher,
                         weight_dict=weight_dict,
                         eos_coef=DefaultConf.eos_coef,
                         losses=DefaultConf.losses
                        )
postprocessors = {"bbox": PostProcess()}

dataset_test = build_dataset(image_set="test", args=DefaultConf)
sample_test = SequentialSampler(dataset_test)
base_ds = get_coco_api_from_dataset(dataset_test)

In [ ]:

basedir = Path("/home/haridas/projects/opensource/detr/")
model_path = basedir / "outputs-2020-06-30-1593500748" / "checkpoint.pth"
state_dict = torch.load(model_path, map_location="cpu")

detr = DETR(backbone=backbone,
            transformer=transformer_network,
            num_queries=100, num_classes=6, aux_loss=False)
detr.load_state_dict(state_dict["model"])
detr.eval();

In [ ]:
data_loader_test = DataLoader(dataset_test,
                              batch_size=2,
                              sampler=sample_test,
                              drop_last=False,
                              collate_fn=collate_fn,
                              num_workers=1)

In [ ]:
# for samples, targets in data_loader_test:
#     print([i['image_id'] for i in targets])
#     import pdb; pdb.set_trace()

In [ ]:
# test_stats, coco_evaluator = evaluate(
#     detr, criterion, postprocessors,
#     data_loader_test,
#     base_ds,
#     device="cpu",
#     output_dir="./out")

# Model Inference

In [ ]:
def load_detr_model(model_path, num_queries=60, num_classes=6):
    basedir = Path(model_path)
    model_path = basedir / "checkpoint.pth"
    state_dict = torch.load(model_path, map_location="cpu")
    detr = DETR(backbone=backbone,
                transformer=transformer_network,
                num_queries=num_queries,
                num_classes=num_classes, aux_loss=False)
    detr.load_state_dict(state_dict["model"])
    detr.eval();
    return detr

In [ ]:
model_path = "/home/haridas/projects/opensource/detr/best_model/checkpoint.pth"
_detr = torch.load(model_path, map_location="cpu")

In [ ]:
_detr['model'].get("transformer.encoder.layers.0.self_attn.in_proj_weight").shape

## Single image Inference

In [ ]:
detr = load_detr_model("/home/haridas/projects/opensource/detr/best_model")

In [ ]:
transform_test = make_coco_transforms("test")

In [ ]:
# output['pred_boxes'][-1, keep]
img = Image.open("/home/haridas/projects/AdaptiveCards-ro/source/pic2card/app/assets/samples/3.png").convert("RGB")
img = Image.open("/home/haridas/projects/mystique/data/templates_test_data/1.png").convert("RGB")
probs, boxes = detect(img, detr_trace_module, transform, threshold=0.8)
scores = probs.max(-1).values.detach().numpy()
classes = probs.max(-1).indices.detach().numpy()
plot_results(img, classes, scores, boxes, label_map=CLASSES, score_threshold=0.8)

In [ ]:
scores.max(-1).values.detach().numpy()

## Using libraries

In [ ]:
from mystique.models.pth.detr.predict import detect as detect_

In [ ]:
img.size

In [ ]:
scores_, boxes_ = detect_(img, detr_trace_module, transform_, threshold=0.8)
plot_results(img, scores_, boxes_, label_map=CLASSES)

In [ ]:
boxes_

## Ploting train vs eval performance

In [ ]:
detr_experiments = [Path(i) for i in glob.glob("/home/haridas/projects/opensource/detr/outputs-2020-07-07*")]

In [ ]:
p = Path("/home/haridas/projects/opensource/detr/best_model/")

In [ ]:
log_df = pd.read_json(p / "log.txt", lines=True)

In [ ]:
# log_df.head().test_coco_eval_bbox[0]

In [ ]:
state_dict = torch.load(p / "checkpoint.pth", map_location="cpu")

In [ ]:
torch.save(state_dict["model"], p / "checkpoint_model.pth")

In [ ]:
# state_dict["model"]

In [ ]:
score = torch.load(p / 'eval.pth')

In [ ]:
score.keys()

In [ ]:
# glob.glob(p / 'eval/*')
plot_precision_recall(
    [Path(p) for p in glob.glob("/home/haridas/projects/opensource/detr/best_model/eval/*.pth")]
     )

In [ ]:
detr_experiments.sort()

In [ ]:
plot_logs(detr_experiments[-1])

# TorchScript 

See how the model can be serialized efficiently for production purpose.

In [ ]:
img = Image.open("/home/haridas/projects/AdaptiveCards-ro/source/pic2card/app/assets/samples/5.png").convert("RGB")
# im = transform(img).unsqueeze(0)

In [ ]:
img_np = np.asarray(img)
im = transform(img).unsqueeze(0)

### Torch Jit Trace

In [ ]:
detr_trace_module = torch.jit.trace(detr, im, strict=False)

In [ ]:
detr_trace_module.save("/home/haridas/projects/pic2card-models/pytorch/detr_trace.pt")

In [ ]:
detr_trace_module = torch.jit.load("/home/haridas/projects/pic2card-models/pytorch/detr_trace.pt")

In [ ]:
t = detr_trace_module(im)

In [ ]:
# print(detr_trace_module.graph)

In [ ]:
from typing import List

@torch.jit.script
def an_error(x):
    #r = torch.rand(1)
    return x

@torch.jit.script
def foo(x, y):
    if x.max() > y.max():
        r = x
    else:
        r = y
    return r

In [ ]:
# print(type(foo))
# print(torch.jit.trace(foo, (torch.ones(2,3), torch.ones(1,2))).code)
# print(foo.code)

In [ ]:
# torch.jit.trace(foo, (torch.ones(2,3), torch.ones(1,2)))

In [ ]:
# print(foo.graph)

### Torch Jit Script

In [ ]:
detr_tscript = torch.jit.script(detr)

In [ ]:
# print(detr_tscript.code)

In [ ]:
detr_tscript.save("/home/haridas/projects/pic2card-models/pytorch/detr.pt")

In [ ]:
# print(detr_tscript.code)

In [ ]:
# !du -sh /home/haridas/projects/pic2card-models/pytorch/detr.pt

In [ ]:
detr_tscript = torch.jit.load("/home/haridas/projects/pic2card-models/pytorch/detr.pt")

In [ ]:
# nested_tensor = NestedTensor(im, None)
# detr_tscript(img)

In [ ]:
# detr_tscript(nested_tensor)
# print(detr_tscript.graph)